In [1]:
import findspark
findspark.init("/usr/local/spark")

ModuleNotFoundError: No module named 'findspark'

In [2]:
import pyspark

ModuleNotFoundError: No module named 'pyspark'

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").config("spark.sql.warehouse.dir", "hdfs://localhost:54310/user/hive/warehouse").enableHiveSupport().getOrCreate()

In [4]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|    capstone|
|   capstones|
|     default|
|       del23|
|       del24|
|    pyspark1|
|    pyspark2|
+------------+



In [5]:
spark.sql("create database pyspark3")

DataFrame[]

In [6]:
spark.sql("use pyspark3")

DataFrame[]

In [7]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [8]:
sc=spark.sparkContext

In [9]:
from pyspark.sql import Row

In [10]:
udata = sc.textFile("/home/hduser/Downloads/pyspark1/u.data")

In [11]:
dffields = udata.map(lambda l: l.split("\t"))

In [18]:
udata = dffields.map(lambda p: Row(UserId=int(p[0]),MovieId=int(p[1]),Rating=int(p[2]),Timestamp=int(p[3])))

In [19]:
udata.take(5)

[Row(MovieId=242, Rating=3, Timestamp=881250949, UserId=196),
 Row(MovieId=302, Rating=3, Timestamp=891717742, UserId=186),
 Row(MovieId=377, Rating=1, Timestamp=878887116, UserId=22),
 Row(MovieId=51, Rating=2, Timestamp=880606923, UserId=244),
 Row(MovieId=346, Rating=1, Timestamp=886397596, UserId=166)]

In [15]:
uuser = sc.textFile("/home/hduser/Downloads/pyspark1/u.user")

In [16]:
uffields = uuser.map(lambda l: l.split("|"))

In [65]:
uuser = uffields.map(lambda p: Row(UserId=int(p[0]),Age=int(p[1]),Gender=p[2],Occupation=p[3],zipcode=p[4]))

In [66]:
uuser.take(5)

[Row(Age=24, Gender='M', Occupation='technician', UserId=1, zipcode='85711'),
 Row(Age=53, Gender='F', Occupation='other', UserId=2, zipcode='94043'),
 Row(Age=23, Gender='M', Occupation='writer', UserId=3, zipcode='32067'),
 Row(Age=24, Gender='M', Occupation='technician', UserId=4, zipcode='43537'),
 Row(Age=33, Gender='F', Occupation='other', UserId=5, zipcode='15213')]

In [67]:
datadf=spark.createDataFrame(udata)

In [68]:
userdf=spark.createDataFrame(uuser)

In [69]:
datadf.write.format("parquet").mode("overwrite").saveAsTable("udata")

In [70]:
userdf.write.format("parquet").mode("overwrite").saveAsTable("userinformation")


In [72]:
spark.sql("show tables").show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|pyspark3|          udata|      false|
|pyspark3|userinformation|      false|
+--------+---------------+-----------+



In [74]:
spark.sql("select * from userinformation order by Gender ").show()

+---+------+-------------+------+-------+
|Age|Gender|   Occupation|UserId|zipcode|
+---+------+-------------+------+-------+
| 30|     F|       artist|    23|  48197|
| 49|     F|    executive|    98|  90291|
| 21|     F|       artist|    24|  94533|
| 53|     F|        other|     2|  94043|
| 40|     F|    librarian|    27|  30030|
| 29|     F|    librarian|    43|  20854|
| 27|     F|    marketing|    46|  46538|
| 39|     F|        other|    11|  30329|
| 23|     F|      student|    49|  76111|
| 49|     F|     educator|    15|  97301|
| 18|     F|      student|    52|  55105|
| 42|     F|    homemaker|    20|  95660|
| 27|     F|administrator|    62|  97214|
| 28|     F|      student|    32|  78741|
| 51|     F|     educator|    65|  48118|
| 20|     F|    homemaker|    35|  42459|
| 48|     F|administrator|    72|  73034|
| 28|     F|        other|    38|  54467|
| 39|     F|administrator|    79|  03755|
| 28|     F|        other|    12|  06405|
+---+------+-------------+------+-

In [81]:
spark.sql("select MovieId,Rating from udata order by Rating desc limit 5").show()

+-------+------+
|MovieId|Rating|
+-------+------+
|     95|     5|
|    201|     5|
|    234|     5|
|    465|     5|
|    603|     5|
+-------+------+



In [78]:
spark.sql("select count(Rating),UserId from udata group by UserId order by count(Rating) desc limit 5").show()

+-------------+------+
|count(Rating)|UserId|
+-------------+------+
|          737|   405|
|          685|   655|
|          636|    13|
|          540|   450|
|          518|   276|
+-------------+------+



In [82]:
spark.sql("select MovieId, Rating from udata order by Rating desc limit 10").show()

+-------+------+
|MovieId|Rating|
+-------+------+
|    603|     5|
|    423|     5|
|    327|     5|
|    222|     5|
|    201|     5|
|    387|     5|
|   1137|     5|
|    234|     5|
|    241|     5|
|      4|     5|
+-------+------+



In [86]:
spark.sql("select MovieId, Rating from udata order by Rating desc limit 9").show()

+-------+------+
|MovieId|Rating|
+-------+------+
|    603|     5|
|    143|     5|
|    327|     5|
|   1014|     5|
|    201|     5|
|    387|     5|
|   1137|     5|
|    234|     5|
|    241|     5|
+-------+------+



In [90]:
spark.sql("select count(Rating),UserId from udata where Rating in (select Rating from udata where Rating=5)group by UserId order by count(Rating) desc limit 10").show()

+-------------+------+
|count(Rating)|UserId|
+-------------+------+
|          172|   416|
|          161|     7|
|          147|    90|
|          145|   592|
|          142|   747|
|          139|   312|
|          137|   551|
|          136|    13|
|          134|    59|
|          126|   450|
+-------------+------+



In [89]:
spark.sql("select UserId,Rating from udata where UserId=416").show()

+------+------+
|UserId|Rating|
+------+------+
|   416|     5|
|   416|     4|
|   416|     4|
|   416|     1|
|   416|     3|
|   416|     5|
|   416|     4|
|   416|     4|
|   416|     5|
|   416|     3|
|   416|     5|
|   416|     1|
|   416|     3|
|   416|     3|
|   416|     4|
|   416|     5|
|   416|     4|
|   416|     4|
|   416|     5|
|   416|     5|
+------+------+
only showing top 20 rows



In [91]:
spark.sql("select count(MovieId) from udata").show()

+--------------+
|count(MovieId)|
+--------------+
|        100000|
+--------------+



In [93]:
userdf.head(10)

[Row(Age=24, Gender='M', Occupation='technician', UserId=1, zipcode='85711'),
 Row(Age=53, Gender='F', Occupation='other', UserId=2, zipcode='94043'),
 Row(Age=23, Gender='M', Occupation='writer', UserId=3, zipcode='32067'),
 Row(Age=24, Gender='M', Occupation='technician', UserId=4, zipcode='43537'),
 Row(Age=33, Gender='F', Occupation='other', UserId=5, zipcode='15213'),
 Row(Age=42, Gender='M', Occupation='executive', UserId=6, zipcode='98101'),
 Row(Age=57, Gender='M', Occupation='administrator', UserId=7, zipcode='91344'),
 Row(Age=36, Gender='M', Occupation='administrator', UserId=8, zipcode='05201'),
 Row(Age=29, Gender='M', Occupation='student', UserId=9, zipcode='01002'),
 Row(Age=53, Gender='M', Occupation='lawyer', UserId=10, zipcode='90703')]

In [103]:
userdf.show()

+---+------+-------------+------+-------+
|Age|Gender|   Occupation|UserId|zipcode|
+---+------+-------------+------+-------+
| 24|     M|   technician|     1|  85711|
| 53|     F|        other|     2|  94043|
| 23|     M|       writer|     3|  32067|
| 24|     M|   technician|     4|  43537|
| 33|     F|        other|     5|  15213|
| 42|     M|    executive|     6|  98101|
| 57|     M|administrator|     7|  91344|
| 36|     M|administrator|     8|  05201|
| 29|     M|      student|     9|  01002|
| 53|     M|       lawyer|    10|  90703|
| 39|     F|        other|    11|  30329|
| 28|     F|        other|    12|  06405|
| 47|     M|     educator|    13|  29206|
| 45|     M|    scientist|    14|  55106|
| 49|     F|     educator|    15|  97301|
| 21|     M|entertainment|    16|  10309|
| 30|     M|   programmer|    17|  06355|
| 35|     F|        other|    18|  37212|
| 40|     M|    librarian|    19|  02138|
| 42|     F|    homemaker|    20|  95660|
+---+------+-------------+------+-

In [102]:
userdf.withColumn("Age Group", userdf.Age/10).show()

+---+------+-------------+------+-------+---------+
|Age|Gender|   Occupation|UserId|zipcode|Age Group|
+---+------+-------------+------+-------+---------+
| 24|     M|   technician|     1|  85711|      2.4|
| 53|     F|        other|     2|  94043|      5.3|
| 23|     M|       writer|     3|  32067|      2.3|
| 24|     M|   technician|     4|  43537|      2.4|
| 33|     F|        other|     5|  15213|      3.3|
| 42|     M|    executive|     6|  98101|      4.2|
| 57|     M|administrator|     7|  91344|      5.7|
| 36|     M|administrator|     8|  05201|      3.6|
| 29|     M|      student|     9|  01002|      2.9|
| 53|     M|       lawyer|    10|  90703|      5.3|
| 39|     F|        other|    11|  30329|      3.9|
| 28|     F|        other|    12|  06405|      2.8|
| 47|     M|     educator|    13|  29206|      4.7|
| 45|     M|    scientist|    14|  55106|      4.5|
| 49|     F|     educator|    15|  97301|      4.9|
| 21|     M|entertainment|    16|  10309|      2.1|
| 30|     M|

In [117]:
userdf1=spark.sql("select *,floor(Age/10) as AgeGroup from userinformation")

In [118]:
userdf1.show()

+---+------+-------------+------+-------+--------+
|Age|Gender|   Occupation|UserId|zipcode|AgeGroup|
+---+------+-------------+------+-------+--------+
| 24|     M|   technician|     1|  85711|       2|
| 53|     F|        other|     2|  94043|       5|
| 23|     M|       writer|     3|  32067|       2|
| 24|     M|   technician|     4|  43537|       2|
| 33|     F|        other|     5|  15213|       3|
| 42|     M|    executive|     6|  98101|       4|
| 57|     M|administrator|     7|  91344|       5|
| 36|     M|administrator|     8|  05201|       3|
| 29|     M|      student|     9|  01002|       2|
| 53|     M|       lawyer|    10|  90703|       5|
| 39|     F|        other|    11|  30329|       3|
| 28|     F|        other|    12|  06405|       2|
| 47|     M|     educator|    13|  29206|       4|
| 45|     M|    scientist|    14|  55106|       4|
| 49|     F|     educator|    15|  97301|       4|
| 21|     M|entertainment|    16|  10309|       2|
| 30|     M|   programmer|    1

In [126]:
userdf1.createOrReplaceTempView("user1")

In [138]:
spark.sql("select *, case when Gender='M' then 1 else 0 end as Male from user1 ").show()

+---+------+-------------+------+-------+--------+----+
|Age|Gender|   Occupation|UserId|zipcode|AgeGroup|Male|
+---+------+-------------+------+-------+--------+----+
| 24|     M|   technician|     1|  85711|       2|   1|
| 53|     F|        other|     2|  94043|       5|   0|
| 23|     M|       writer|     3|  32067|       2|   1|
| 24|     M|   technician|     4|  43537|       2|   1|
| 33|     F|        other|     5|  15213|       3|   0|
| 42|     M|    executive|     6|  98101|       4|   1|
| 57|     M|administrator|     7|  91344|       5|   1|
| 36|     M|administrator|     8|  05201|       3|   1|
| 29|     M|      student|     9|  01002|       2|   1|
| 53|     M|       lawyer|    10|  90703|       5|   1|
| 39|     F|        other|    11|  30329|       3|   0|
| 28|     F|        other|    12|  06405|       2|   0|
| 47|     M|     educator|    13|  29206|       4|   1|
| 45|     M|    scientist|    14|  55106|       4|   1|
| 49|     F|     educator|    15|  97301|       

In [139]:
userdf2=spark.sql("select *, case when Gender='M' then 1 else 0 end as Male from user1 ")

In [140]:
userdf2.show()

+---+------+-------------+------+-------+--------+----+
|Age|Gender|   Occupation|UserId|zipcode|AgeGroup|Male|
+---+------+-------------+------+-------+--------+----+
| 24|     M|   technician|     1|  85711|       2|   1|
| 53|     F|        other|     2|  94043|       5|   0|
| 23|     M|       writer|     3|  32067|       2|   1|
| 24|     M|   technician|     4|  43537|       2|   1|
| 33|     F|        other|     5|  15213|       3|   0|
| 42|     M|    executive|     6|  98101|       4|   1|
| 57|     M|administrator|     7|  91344|       5|   1|
| 36|     M|administrator|     8|  05201|       3|   1|
| 29|     M|      student|     9|  01002|       2|   1|
| 53|     M|       lawyer|    10|  90703|       5|   1|
| 39|     F|        other|    11|  30329|       3|   0|
| 28|     F|        other|    12|  06405|       2|   0|
| 47|     M|     educator|    13|  29206|       4|   1|
| 45|     M|    scientist|    14|  55106|       4|   1|
| 49|     F|     educator|    15|  97301|       

In [141]:
userdf2.createOrReplaceTempView("user2")

In [142]:
spark.sql("select *, case when Gender='F' then 1 else 0 end as Female from user2 ").show()

+---+------+-------------+------+-------+--------+----+------+
|Age|Gender|   Occupation|UserId|zipcode|AgeGroup|Male|Female|
+---+------+-------------+------+-------+--------+----+------+
| 24|     M|   technician|     1|  85711|       2|   1|     0|
| 53|     F|        other|     2|  94043|       5|   0|     1|
| 23|     M|       writer|     3|  32067|       2|   1|     0|
| 24|     M|   technician|     4|  43537|       2|   1|     0|
| 33|     F|        other|     5|  15213|       3|   0|     1|
| 42|     M|    executive|     6|  98101|       4|   1|     0|
| 57|     M|administrator|     7|  91344|       5|   1|     0|
| 36|     M|administrator|     8|  05201|       3|   1|     0|
| 29|     M|      student|     9|  01002|       2|   1|     0|
| 53|     M|       lawyer|    10|  90703|       5|   1|     0|
| 39|     F|        other|    11|  30329|       3|   0|     1|
| 28|     F|        other|    12|  06405|       2|   0|     1|
| 47|     M|     educator|    13|  29206|       4|   1|

In [143]:
userdf3=spark.sql("select *, case when Gender='F' then 1 else 0 end as Female from user2 ")

In [144]:
userdf3.show()

+---+------+-------------+------+-------+--------+----+------+
|Age|Gender|   Occupation|UserId|zipcode|AgeGroup|Male|Female|
+---+------+-------------+------+-------+--------+----+------+
| 24|     M|   technician|     1|  85711|       2|   1|     0|
| 53|     F|        other|     2|  94043|       5|   0|     1|
| 23|     M|       writer|     3|  32067|       2|   1|     0|
| 24|     M|   technician|     4|  43537|       2|   1|     0|
| 33|     F|        other|     5|  15213|       3|   0|     1|
| 42|     M|    executive|     6|  98101|       4|   1|     0|
| 57|     M|administrator|     7|  91344|       5|   1|     0|
| 36|     M|administrator|     8|  05201|       3|   1|     0|
| 29|     M|      student|     9|  01002|       2|   1|     0|
| 53|     M|       lawyer|    10|  90703|       5|   1|     0|
| 39|     F|        other|    11|  30329|       3|   0|     1|
| 28|     F|        other|    12|  06405|       2|   0|     1|
| 47|     M|     educator|    13|  29206|       4|   1|

In [145]:
userdf3.createOrReplaceTempView('user3')

In [174]:
spark.sql("select distinct(Occupation) from user2").show()

+-------------+
|   Occupation|
+-------------+
|    librarian|
|      retired|
|       lawyer|
|         none|
|       writer|
|   programmer|
|    marketing|
|        other|
|    executive|
|    scientist|
|      student|
|     salesman|
|       artist|
|   technician|
|administrator|
|     engineer|
|   healthcare|
|     educator|
|entertainment|
|    homemaker|
+-------------+
only showing top 20 rows



In [178]:
spark.sql("select *, dense_rank() over (order by seqnum) as OccupationId from (select *, min(UserId) over (partition by Occupation) as seqnum from user3) user3 order by UserId").show()

+---+------+-------------+------+-------+--------+----+------+------+------------+
|Age|Gender|   Occupation|UserId|zipcode|AgeGroup|Male|Female|seqnum|OccupationId|
+---+------+-------------+------+-------+--------+----+------+------+------------+
| 24|     M|   technician|     1|  85711|       2|   1|     0|     1|           1|
| 53|     F|        other|     2|  94043|       5|   0|     1|     2|           2|
| 23|     M|       writer|     3|  32067|       2|   1|     0|     3|           3|
| 24|     M|   technician|     4|  43537|       2|   1|     0|     1|           1|
| 33|     F|        other|     5|  15213|       3|   0|     1|     2|           2|
| 42|     M|    executive|     6|  98101|       4|   1|     0|     6|           4|
| 57|     M|administrator|     7|  91344|       5|   1|     0|     7|           5|
| 36|     M|administrator|     8|  05201|       3|   1|     0|     7|           5|
| 29|     M|      student|     9|  01002|       2|   1|     0|     9|           6|
| 53

In [183]:
userdf4=spark.sql("select user3.*, dense_rank() over (order by seqnum) as OccupationId from (select user3.*, min(UserId) over (partition by Occupation) as seqnum from user3) user3 order by UserId")

In [184]:
userdf4.show()

+---+------+-------------+------+-------+--------+----+------+------+------------+
|Age|Gender|   Occupation|UserId|zipcode|AgeGroup|Male|Female|seqnum|OccupationId|
+---+------+-------------+------+-------+--------+----+------+------+------------+
| 24|     M|   technician|     1|  85711|       2|   1|     0|     1|           1|
| 53|     F|        other|     2|  94043|       5|   0|     1|     2|           2|
| 23|     M|       writer|     3|  32067|       2|   1|     0|     3|           3|
| 24|     M|   technician|     4|  43537|       2|   1|     0|     1|           1|
| 33|     F|        other|     5|  15213|       3|   0|     1|     2|           2|
| 42|     M|    executive|     6|  98101|       4|   1|     0|     6|           4|
| 57|     M|administrator|     7|  91344|       5|   1|     0|     7|           5|
| 36|     M|administrator|     8|  05201|       3|   1|     0|     7|           5|
| 29|     M|      student|     9|  01002|       2|   1|     0|     9|           6|
| 53

In [190]:
df1=spark.sql("select user3.*, min(UserId) over (partition by Occupation) as seqnum from user3 order by UserId")

In [191]:
df1.createOrReplaceTempView('df1')

In [192]:
spark.sql("select * from df1 where seqnum=8").show()

+---+------+----------+------+-------+--------+----+------+------+
|Age|Gender|Occupation|UserId|zipcode|AgeGroup|Male|Female|seqnum|
+---+------+----------+------+-------+--------+----+------+------+
+---+------+----------+------+-------+--------+----+------+------+



In [193]:
userdf4.createOrReplaceTempView('user4')

In [194]:
spark.sql("select * from user4 where OccupationId=8").show()

+---+------+----------+------+-------+--------+----+------+------+------------+
|Age|Gender|Occupation|UserId|zipcode|AgeGroup|Male|Female|seqnum|OccupationId|
+---+------+----------+------+-------+--------+----+------+------+------------+
| 47|     M|  educator|    13|  29206|       4|   1|     0|    13|           8|
| 49|     F|  educator|    15|  97301|       4|   0|     1|    13|           8|
| 28|     M|  educator|    51|  16509|       2|   1|     0|    13|           8|
| 49|     M|  educator|    59|  08403|       4|   1|     0|    13|           8|
| 32|     M|  educator|    64|  43202|       3|   1|     0|    13|           8|
| 51|     F|  educator|    65|  48118|       5|   0|     1|    13|           8|
| 51|     M|  educator|    85|  20003|       5|   1|     0|    13|           8|
| 60|     M|  educator|    90|  78155|       6|   1|     0|    13|           8|
| 44|     M|  educator|   108|  63130|       4|   1|     0|    13|           8|
| 50|     M|  educator|   137|  84408|  

In [198]:
userdf5=userdf4.drop('seqnum')

In [200]:
userdf5.createOrReplaceTempView('user5')

In [202]:
spark.sql("select * from user5").show()

+---+------+-------------+------+-------+--------+----+------+------------+
|Age|Gender|   Occupation|UserId|zipcode|AgeGroup|Male|Female|OccupationId|
+---+------+-------------+------+-------+--------+----+------+------------+
| 24|     M|   technician|     1|  85711|       2|   1|     0|           1|
| 53|     F|        other|     2|  94043|       5|   0|     1|           2|
| 23|     M|       writer|     3|  32067|       2|   1|     0|           3|
| 24|     M|   technician|     4|  43537|       2|   1|     0|           1|
| 33|     F|        other|     5|  15213|       3|   0|     1|           2|
| 42|     M|    executive|     6|  98101|       4|   1|     0|           4|
| 57|     M|administrator|     7|  91344|       5|   1|     0|           5|
| 36|     M|administrator|     8|  05201|       3|   1|     0|           5|
| 29|     M|      student|     9|  01002|       2|   1|     0|           6|
| 53|     M|       lawyer|    10|  90703|       5|   1|     0|           7|
| 39|     F|